案例：探究用户对物品类别的喜好细分


  1.   用户  物品类别 

  2.  user_id  aisle 
    
    
    

* 需要将user_id和aisle放在同一个表中 - 合并
* 找到user_id和aisle - 交叉表和透视表
* 特征冗余过多 -> PCA降维

In [1]:
import pandas as  pd
# 1.获取数据
order_products = pd.read_csv("../../resource/instacart/order_products__prior.csv")
products = pd.read_csv("../../resource/instacart/products.csv")
orders = pd.read_csv("../../resource/instacart/orders.csv")
aisles = pd.read_csv("../../resource/instacart/aisles.csv")


In [2]:
#2.处理数据
#  1）合并表
tab1 = pd.merge(aisles, products, on=["aisle_id", "aisle_id"])
tab2 = pd.merge(tab1, order_products, on=["product_id", "product_id"])
tab3 = pd.merge(tab2, orders, on=["order_id", "order_id"])

In [3]:
# 2）.找到user和aisle 之间的关系 列user_id 行aisle
table = pd.crosstab(tab3["user_id"], tab3["aisle"])

In [4]:
table.head()

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,3,0,0,0,0,2,0,0,0,...,3,1,1,0,0,0,0,2,0,42
3,0,0,0,0,0,0,0,0,0,0,...,4,1,0,0,0,0,0,2,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# 3).PAC降维处理
# 3.1)实例化一个转换器 n_components=0.95降维保留95%的数据，n_components=2 降为2维
transform = PCA(n_components=0.95)
# 3.2)调用fit_transform
data_new = transform.fit_transform(table)

In [6]:
data_new = data_new[:1000000]

In [7]:
# 预估器流程
from sklearn.cluster import KMeans
estimator = KMeans(n_clusters=3)
estimator.fit(data_new)
y_predict = estimator.predict(data_new)

In [8]:
y_predict[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [73]:
# 模型评估-轮廓系数 [-1,1] -1效果最不好  1效果最好
from sklearn.metrics import silhouette_score

In [74]:
score = silhouette_score(data_new,y_predict)

In [75]:
print(score)

0.5374100487431882
